### Read config file

In [1]:
import numpy as np
import obspy
from obspy import UTCDateTime
from ccfj import CC
from ccfj import GetStationPairs
from concurrent.futures import ThreadPoolExecutor
import os
import time
from geopy.distance import great_circle
import folium
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import yaml
import shutil

In [2]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib

In [3]:
with open('a-project.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project/output_FJSJ_16-01/'               # Harmon server
name_project

'project/output_FJSJ_v9.1_test/'

In [4]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = dir_project_workspace + name_project
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project/output_FJSJ_v9.1_test/


In [5]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

### parameter settings

In [6]:
tag = info_basic['tag']
stalistname_all = info_basic['stalistname_all']

### Generat sta_info

In [7]:
stainfo = pd.read_excel(stalistname_all)
nsta = len(stainfo.iloc[:,0])
StationPairs = GetStationPairs(nsta)
nPairs = int(len(StationPairs)/2)
stalist = stainfo['Station'].tolist()
lat = stainfo['latitude'].tolist() 
lon = stainfo['longitude'].tolist()

In [8]:
sta_info = {}
years = {2014}
for i in range(nsta):
    sta_info[stalist[i]] = {}
    sta_info[stalist[i]]['year'] = 2014
    sta_info[stalist[i]]['lat'] =float(lat[i])
    sta_info[stalist[i]]['lon'] =float(lon[i])

In [9]:
for key in list(sta_info.keys()):
    year = sta_info[key]['year']
    sta_info[key]['d_start'] = info_basic['d_start']
    sta_info[key]['d_end'] = info_basic['d_end']

### Sort by different years

In [10]:
cs = maplib.cs
crs = maplib.crs
zoom_rate = 16

In [11]:
key_loc = 12
this_map = maplib.plot_single_area(sta_info,key_loc,cs,crs,zoom_rate)
this_map

### Sort area automatically with row and column indexes

In [ ]:
def sort_1d(sta_info,len_col,step_col,inter_col,flag_col,flag_row):
    print("temporarily not available")
    rows = np.arange(int(flag_row[0]),int(flag_row[1])+1)
    cols = np.arange(int(flag_col[0]),int(flag_col[1])+1,inter_col)

    sta_area = {}
    for row in rows:
        flag_col = 0
        flag_col_tag = 0
        while flag_col + len_col <  len(cols):
            flag_col_tag += 1
            key = str(row).zfill(2)+'-'+str(flag_col_tag).zfill(2)
            sta_area[key] = {}
            for i in range(len_col):
                for j in range(len_col):
                    name = 'R'+str(cols[flag_col+j]).zfill(2)+str(row).zfill(2)
                    sta_area[key][name] ={}
                    sta_area[key][name]['lat'] = sta_info[name]['lat']
                    sta_area[key][name]['lon'] = sta_info[name]['lon']
            flag_col += step_col
    return sta_area

def sort_2d(sta_info,inter_row,inter_col,size_row,size_col,step_row,step_col,tag_col,tag_row,stas):
    rows = np.arange(int(tag_row[0]),int(tag_row[1])+1,inter_row)
    cols = np.arange(int(tag_col[0]),int(tag_col[1])+1,inter_col)

    sta_area = {}
    flag_row = 0

    flag_row_tag = 0
    while flag_row + size_row <= len(rows):
        flag_col = 0
        flag_row_tag += 1
        flag_col_tag = 0
        while flag_col + size_col <=  len(cols):
            flag_col_tag += 1
            key = str(flag_row_tag).zfill(2)+'-'+str(flag_col_tag).zfill(2)
            sta_area[key] = {}
            for i in range(size_row):
                for j in range(size_col):
                    name = 'R'+str(cols[flag_col+j]).zfill(2)+str(rows[flag_row+i]).zfill(2)
                    if name in stas:
                        sta_area[key][name] ={}
                        sta_area[key][name]['lat'] = sta_info[name]['lat']
                        sta_area[key][name]['lon'] = sta_info[name]['lon']
            flag_col += step_col
        flag_row += step_row

    return sta_area

In [ ]:
def sort_point_2d(sta_info,stas,step_col,step_row,size_col,size_row,tag_col,tag_row):
    rows = np.arange(int(tag_row[0]),int(tag_row[1])+1)
    cols = np.arange(int(tag_col[0]),int(tag_col[1])+1)

    len_row = int((size_row-1)/2)
    len_col = int((size_col-1)/2)

    sta_area = {}
    flag_row = tag_row[0]

    while flag_row + len_row <= tag_row[1]:
        while flag_row - len_row <=  tag_row[0]-1:
            flag_row += 1
        flag_col = tag_col[0]
        while flag_col + len_col <= tag_col[1]:
            while flag_col - len_col <= tag_col[0]-1:
                flag_col += 1
            
            key = str(flag_col).zfill(2)+'-'+str(flag_row).zfill(2)
            
            sta_area[key] = {}
            for i in range(flag_row-len_row-1,flag_row+len_row):
                for j in range(flag_col-len_col-1,flag_col+len_col):
                    name = 'R'+str(cols[j]).zfill(2)+str(rows[i]).zfill(2)
                    if name in stas:
                        sta_area[key][name] ={}
                        sta_area[key][name]['lat'] = sta_info[name]['lat']
                        sta_area[key][name]['lon'] = sta_info[name]['lon']
            flag_col += step_col
        flag_row += step_row
    return sta_area

In [ ]:
def sort_point_2d_all(sta_info,stas,step_col,step_row,size_col,size_row,tag_col,tag_row):
    rows = np.arange(int(tag_row[0]),int(tag_row[1])+1)
    cols = np.arange(int(tag_col[0]),int(tag_col[1])+1)
    
    sta_area = {}
    flag_row = tag_row[0]

    while flag_row + size_row <= tag_row[1]:
        flag_col = tag_col[0]
        while flag_col + size_col <= tag_col[1]:
            key = str(flag_col).zfill(2)+'-'+str(flag_row).zfill(2)
            sta_area[key] = {}
            for i in range(size_row):
                for j in range(size_col):
                    name = 'R'+str(cols[flag_col+j]).zfill(2)+str(rows[flag_row+i]).zfill(2)
                    if name in stas:
                        sta_area[key][name] ={}
                        sta_area[key][name]['lat'] = sta_info[name]['lat']
                        sta_area[key][name]['lon'] = sta_info[name]['lon']
            flag_col += step_col

    return sta_area

In [ ]:
"""
len_col = 15
step_col = 10
inter_col = 1

flag_row = [1,20]
flag_col = [11,60]

sta_area = sort_1d(sta_info,len_col,step_col,inter_col,flag_col,flag_row)
"""

In [ ]:
"""
inter_row = 1
inter_col = 1

size_row = 5    # 5
size_col = 15    # 15

step_row = 5
step_col = 15

tag_row = [1,20]
tag_col = [1,60]
#tag_row = [1,6]
#tag_col = [11,26]

stas = list(sta_info.keys())

sta_area = sort_2d(sta_info,inter_row,inter_col,size_row,size_col,step_row,step_col,tag_col,tag_row,stas)
"""

In [ ]:
step_row = 10
step_col = 10

size_row = 5
size_col = 13

tag_row = [1,20]
tag_col = [1,60]

stas = list(sta_info.keys())

sta_area = sort_point_2d(sta_info,stas,step_col,step_row,size_col,size_row,tag_col,tag_row)


In [ ]:
this_map = maplib.plot_single_area(sta_info,0,cs,crs,zoom_rate)
i = 2
key = list(sta_area.keys())[i]
#key = '13-04'
this_map = maplib.lat_lon(sta_area[key],cs[np.mod(i+1,2)],crs[np.mod(i,2)],this_map)
this_map

In [ ]:
this_map = maplib.plot_single_area(sta_info,10,cs,crs,zoom_rate)
for i in range(len(sta_area.keys())):
    key = list(sta_area.keys())[i]
    this_map = maplib.lat_lon(sta_area[key],cs[np.mod(i,4)],crs[np.mod(i+1,3)],this_map)
this_map

### Generate subworks

In [ ]:
key_subworks = list(sta_area.keys())
info_basic['key_subworks'] = list(sta_area.keys())
len(key_subworks)

In [ ]:
dir_partition = dir_project + 'partition/'
info_basic['dir_partition'] = 'partition/'
if os.path.exists(dir_partition):
    shutil.rmtree(dir_partition)
os.makedirs(dir_partition)

In [ ]:
nstaS = {}
for key_subwork in key_subworks:
    nstaS[key_subwork] = len(list(sta_area[key_subwork].keys()))
    Station = list(sta_area[key_subwork].keys())
    lat = [sta_area[key_subwork][key]['lat'] for key in list(sta_area[key_subwork].keys())]
    lon = [sta_area[key_subwork][key]['lon'] for key in list(sta_area[key_subwork].keys())]
    filename = dir_partition + key_subwork + '.txt'
    np.savetxt(filename, np.array([Station, lat, lon]).T, fmt='%s', delimiter=' ', header='Station lat lon')
info_basic['nstaS'] = nstaS

In [ ]:
#np.save(info_basic['dir_config']+'/Basic_info.npy', info_basic)
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)